In [1]:
import pandas as pd
from scipy import stats
import numpy as np

### Data Upload

In [2]:
# R1_19, L1_18 and R2_16 are gas sensors; Atmos2, Atmos3 and Atmos6 are particulate matter(PM) sensors.
R1_19=pd.read_excel('May04_Day02_SMShetty_HSSSensor_Rawdata.xlsx',sheet_name='R1_19',usecols= ['DateTime','NO','NO2','CO','CO2','ExtTemp'],na_values=['NaN'])
L1_18=pd.read_excel('May04_Day02_SMShetty_HSSSensor_Rawdata.xlsx',sheet_name='L1_18',usecols= ['DateTime','NO','NO2','CO','CO2','ExtTemp'],na_values=['NaN'])
R2_16=pd.read_excel('May04_Day02_SMShetty_HSSSensor_Rawdata.xlsx', sheet_name='R2_16',usecols= ['DateTime','NO','NO2','CO','CO2','ExtTemp'],na_values=['NaN'])

Atmos2=pd.read_excel('Atmos2_API_SD_May04_FreqMinute.xlsx',na_values=['NaN'])
Atmos3=pd.read_excel('Atmos3_API_May04_Freq05Second.xlsx',na_values=['NaN'])
Atmos6=pd.read_excel('Atmos6_API_SD_May04_Freq05seconds.xlsx',na_values=['NaN'])

R1_19['NOx']=R1_19['NO']+R1_19['NO2']
L1_18['NOx']=L1_18['NO']+L1_18['NO2']
R2_16['NOx']=R2_16['NO']+R2_16['NO2']

### Defining Time

In [3]:
Gas_sensors=[R1_19,L1_18,R2_16]
PM_sensors=[Atmos2,Atmos3,Atmos6]
for i in Gas_sensors:
    i['TimeDate'] = pd.to_datetime(i['DateTime'])
    i['hour']=i['TimeDate'].dt.hour
    i['day']=i['TimeDate'].dt.day
    
for i in PM_sensors:
    i['TimeDate'] = pd.to_datetime(i['dt_time'])
    i['hour']=i['TimeDate'].dt.hour
    i['day']=i['TimeDate'].dt.day

### Hourly Mean

In [4]:
hourly_mean_R1_19=R1_19.groupby(['hour'])['CO','NOx','CO2'].mean()
hourly_mean_R2_16=R2_16.groupby(['hour'])['CO','NOx','CO2'].mean()
hourly_mean_L1_18=L1_18.groupby(['hour'])['CO','NOx','CO2'].mean()
hourly_mean_Atmos2=Atmos2.groupby(['hour'])['pm25raw','pm1cnc','pm10cnc'].mean()
hourly_mean_Atmos3=Atmos3.groupby(['hour'])['pm25raw','pm1cnc','pm10cnc'].mean()
hourly_mean_Atmos6=Atmos6.groupby(['hour'])['pm25raw','pm1cnc','pm10cnc'].mean()

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_9024/327505495.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_mean_R1_19=R1_19.groupby(['hour'])['CO','NOx','CO2'].mean()
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_9024/327505495.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_mean_R2_16=R2_16.groupby(['hour'])['CO','NOx','CO2'].mean()
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_9024/327505495.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_mean_L1_18=L1_18.groupby(['hour'])['CO','NOx','CO2'].mean()
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_9024/327505495.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_mean_Atmos2=Atmos2.g

### Mean of All sensors

In [5]:
### Gases##
df_CO = pd.concat((hourly_mean_R1_19['CO'], hourly_mean_R2_16['CO'],hourly_mean_L1_18['CO']), axis=1) # Concating hourly mean CO from all sensors
df_NOx = pd.concat((hourly_mean_R1_19['NOx'], hourly_mean_R2_16['NOx'],hourly_mean_L1_18['NOx']), axis=1)# Concating hourly mean NOx from all sensors
df_CO2 = pd.concat((hourly_mean_R1_19['CO2'], hourly_mean_R2_16['CO2'],hourly_mean_L1_18['CO2']), axis=1)# Concating hourly mean CO2 from all sensors

CO_mean=df_CO.mean(axis=1) # Hourly_Mean of each hourly_mean from all sensors.
NOx_mean=df_NOx.mean(axis=1)
CO2_mean=df_CO2.mean(axis=1)

### PM ###
df_PM25 = pd.concat((hourly_mean_Atmos2['pm25raw'], hourly_mean_Atmos3['pm25raw'],hourly_mean_Atmos6['pm25raw']), axis=1)# Concating hourly mean PM2.5 from all sensors
df_PM1 = pd.concat((hourly_mean_Atmos2['pm1cnc'], hourly_mean_Atmos3['pm1cnc'],hourly_mean_Atmos6['pm1cnc']), axis=1)
df_PM10 = pd.concat((hourly_mean_Atmos2['pm10cnc'], hourly_mean_Atmos3['pm10cnc'],hourly_mean_Atmos6['pm10cnc']), axis=1)

PM25_mean=df_PM25.mean(axis=1)
PM1_mean=df_PM1.mean(axis=1)
PM10_mean=df_PM10.mean(axis=1)

### Saving into single excel file

In [6]:
with pd.ExcelWriter('May04_Hourly_mean_Allsensors_ideal.xlsx') as writer1:
    CO_mean.to_excel(writer1, sheet_name = 'CO', index = True)
    NOx_mean.to_excel(writer1, sheet_name = 'NOx', index = True)
    CO2_mean.to_excel(writer1, sheet_name = 'CO2', index = True)
    PM25_mean.to_excel(writer1, sheet_name = 'PM2.5', index = True)
    PM1_mean.to_excel(writer1, sheet_name = 'PM1', index = True)
    PM10_mean.to_excel(writer1, sheet_name = 'PM10', index = True)